In [1]:
import tweepy
import pandas as pd

In [9]:
def get_all_tweets(screen_name,since_id,consumer_key,consumer_secret,access_key,access_secret):
    #Twitter only allows access to a users most recent 3240 tweets with this method
    
    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)

    #initialize a list to hold all the tweepy Tweets
    alltweets = []	
    
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200, since_id = since_id)
    
    #save most recent tweets
    alltweets.extend(new_tweets)
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        #print "getting tweets before %s" % (oldest)
        
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest, since_id = since_id)
    
        #save most recent tweets
        alltweets.extend(new_tweets)
        
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        
        #print "...%s tweets downloaded so far" % (len(alltweets))
    #transform the tweepy tweets into a 2D array

    outtweets = [[tweet.created_at, tweet.text.encode("utf-8"), tweet.source, tweet.favorite_count, tweet.retweet_count,
                  [p['text'] for p in tweet.entities['hashtags']], [p['name'] for p in tweet.entities['user_mentions']],
                  (tweet.entities['urls']!=[]), ('media' in tweet.entities.keys())] for tweet in alltweets]
    result = pd.DataFrame(outtweets)
    result.columns = ["created_time","text","source","favorite_count","retweet_count",'hashtags','user_mentioned',
                      'url_used','media_used']
    
    return result

In [7]:
consumer_key = '1ZvRTduPL1QaxWw1RAYW9UKDS'
consumer_secret = '0gwH9jmJlZfmQWPBWne7vFIQD6AJBHiPSJpMINdSouvrJ17SNC'
access_token_key =  '836992991814246400-RCdOn9BxTzzrH2wDZoZQT8jiKj7SP3P'
access_token_secret = 'MkziDeQ6kvaBYdiaGeoEgqrTj6GHJNWqYph3B8yQJgdjT'